In [ ]:
region = "2RL:63,000,000-65,000,000"
region_name = "the Vgsc"
contig, span = region.split(":")
dask_scheduler = "threads"
use_gcs_cache = False
cohorts_analysis = "20230223"
analysis_version = "dev"

In [ ]:
from IPython.display import Markdown, HTML
import malariagen_data
import numpy as np
import pandas as pd
from pyprojroot import here
import yaml
import dask

dask.config.set(scheduler=dask_scheduler)
from textwrap import dedent

import geopandas as gpd
import bokeh.layouts as bklay
import bokeh.plotting as bkplt
import bokeh.models as bkmod
import plotly.express as px

from bokeh.io import output_notebook  # enables plot interface in J notebook

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# SA-1

## Signals

In [ ]:
extra_params = dict()
if use_gcs_cache:
    extra_params["url"] = "simplecache::gs://vo_agam_release_master_us_central1"
    extra_params["simplecache"] = dict(cache_storage=(here() / "gcs_cache").as_posix())

ag3 = malariagen_data.Ag3(
    # pin the version of the cohorts analysis for reproducibility
    cohorts_analysis=cohorts_analysis,
    results_cache=(here() / "build" / "malariagen_data_cache").as_posix(),
    **extra_params,
)
ag3

In [ ]:
def load_signals(cohorts, contig):
    df_signals = []
    for _, row in cohorts.iterrows():
        try:
            df_signals.append(
                pd.read_csv(
                    here()
                    / "build/h12-signal-detection/"
                    / f"{row['cohort_id']}_{contig}.csv"
                ).assign(taxon=row["taxon"])
            )
        except pd.errors.EmptyDataError:
            continue

    df_signals = (
        pd.concat(df_signals, axis=0).assign(statistic="H12").sort_values("taxon")
    )
    color_dict = {"gambiae": "#BEC4FF", "coluzzii": "#D7B2A6", "arabiensis": "#A6D7CA"}

    df_signals["color"] = df_signals["taxon"].map(color_dict).fillna("lightgrey")

    return df_signals


cohorts = gpd.read_file(here() / "build" / "final_cohorts.geojson")
df_signals = load_signals(cohorts=cohorts, contig=contig)

start, stop = span.replace(",", "").split("-")
region_signals = df_signals.query(
    f"focus_pstop < {int(stop)} and focus_pstart > {int(start)}"
).sort_values("cohort_id")
region_signals = region_signals.merge(cohorts)

In [ ]:
def stack_overlaps(df, start_col, end_col, tolerance=10000):
    import numpy as np

    occupants = [None]
    out = []
    for _, cur in df.iterrows():
        level = 0
        prv = occupants[level]
        # search upwards to find the first vacant level
        while prv is not None and cur[start_col] <= (prv[end_col] + tolerance):
            level += 1
            if level == len(occupants):
                occupants.append(None)
            prv = occupants[level]
        occupants[level] = cur
        out.append(level)
    return np.asarray(out)


region_signals = region_signals.sort_values(by="span2_pstart")
region_signals["level"] = stack_overlaps(region_signals, "span2_pstart", "span2_pstop")

In [ ]:
def plot_region_summary(df, x_min, x_max, region_name):
    """
    This function is the same as the plot_chromosome_summary() in the chromosome pages notebook, but sets the
    x_range to be the left-most signal -> right-most signal, rather than the start and end of the chromosome.
    """
    x_min = df["span2_pstart"].min() - 500_000
    x_max = df["span2_pstop"].max() + 500_000

    # set up triangle shapes for bokeh patches glyphs
    left_xs = [
        np.array([row.span2_pstart, row.focus_pstart, row.focus_pstart])
        for idx, row in df.iterrows()
    ]
    left_ys = [
        np.array([row.level + 0.1, row.level, row.level + 0.2])
        for idx, row in df.iterrows()
    ]

    right_xs = [
        np.array([row.focus_pstop, row.focus_pstop, row.span2_pstop])
        for idx, row in df.iterrows()
    ]
    right_ys = [
        np.array([row.level, row.level + 0.2, row.level + 0.1])
        for idx, row in df.iterrows()
    ]

    center_xs = [np.array([row.pcenter, row.pcenter]) for idx, row in df.iterrows()]
    center_ys = [np.array([row.level, row.level + 0.2]) for idx, row in df.iterrows()]

    source = bkmod.ColumnDataSource(
        data={
            "cohort": df.cohort_id,
            "taxon": df.taxon,
            "statistic": df.statistic,
            "chromosome": df.contig,
            "score": df.delta_i.astype(int),
            "peak_start": df.span2_pstart,
            "peak_stop": df.span2_pstop,
            "focus_start": df.focus_pstart,
            "focus_stop": df.focus_pstop,
            "left_xs": left_xs,
            "left_ys": left_ys,
            "right_xs": right_xs,
            "right_ys": right_ys,
            "center_xs": center_xs,
            "center_ys": center_ys,
            "bottom": df.level,
            "mid": df.level + 0.5,
            "top": df.level + 0.2,
            "taxon_color": df.color,
        }
    )

    hover = bkmod.HoverTool(
        tooltips=[
            ("Cohort", "@cohort"),
            ("Statistic", "@statistic"),
            ("Score", "@score"),
            ("Focus", "@focus_start{,} - @focus_stop{,}"),
        ]
    )

    xwheel_zoom = bkmod.WheelZoomTool(dimensions="width", maintain_focus=False)

    # make figure
    fig1 = bkplt.figure(
        title=f"Selection signals at {region_name}",
        width=900,
        height=200 + (10 * max(df.level)),
        tools=["tap", "xpan", "xzoom_in", "xzoom_out", "reset", xwheel_zoom, hover],
        toolbar_location="above",
        active_drag="xpan",
        x_range=bkmod.Range1d(x_min, x_max, bounds="auto"),
        y_range=bkmod.Range1d(-0.5, max(df.level) + 1.3, bounds="auto"),
        active_scroll=xwheel_zoom,
    )

    fig1.patches(
        xs="left_xs",
        ys="left_ys",
        source=source,
        color="taxon_color",
        alpha=0.7,
        line_width=2,
        legend_field="taxon",
    )
    fig1.patches(
        xs="right_xs",
        ys="right_ys",
        source=source,
        color="taxon_color",
        alpha=0.7,
        line_width=2,
        legend_field="taxon",
    )

    fig1.quad(
        bottom="bottom",
        top="top",
        left="focus_start",
        right="focus_stop",
        source=source,
        color="red",
        alpha=0.5,
        line_width=2,
    )

    glyph = bkmod.MultiLine(
        xs="center_xs", ys="center_ys", line_color="red", line_width=2, line_alpha=0.8
    )
    fig1.add_glyph(source, glyph)

    # tidy up the plot
    fig1.yaxis.visible = False
    fig1.xaxis.visible = False
    fig1.ygrid.visible = False
    fig1.legend.background_fill_alpha = 0.2

    url = "../cohort/@cohort.html"
    taptool = fig1.select(type=bkmod.TapTool)
    taptool.callback = bkmod.OpenURL(url=url)

    fig2 = ag3.plot_genes(
        region=contig, sizing_mode="stretch_width", x_range=fig1.x_range, show=False
    )

    fig = bklay.gridplot(
        [fig1, fig2],
        ncols=1,
        toolbar_location="above",
        merge_tools=True,
        sizing_mode="stretch_width",
    )

    bkplt.show(fig)


plot_region_summary(df=region_signals.reset_index(), region_name=region_name)

## Cohorts affected
Overlapping signals of selection are found in the following cohorts. 

In [ ]:
cohort_links = [
    '<a href="../cohort/' + row["cohort_id"] + '.html">' + row["cohort_label"] + "</a>"
    for i, row in region_signals.iterrows()
]
html_message = "<li>" + "</li><li>".join(cohort_links) + "</li>"
HTML(html_message)

## Candidate genes

AGAP004707 (*Vgsc*) - The Voltage-gated sodium channel (VGSC) is the target of pyrethroids and DDT. Mutations in this gene are associated with knockdown resistance (kdr) to pyrethroids and DDT...

## Change log
- 2000-00-00 - first version published